[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/notebooks/notebook_04_implementacion_arboles-published.ipynb)

## Árboles de decisión

### Metiendonos debajo del capot

En esta notebook exploraremos el funcionamiento de un árbol de decisión construido aquí mismo.

Para eso contaremos con algunas partes de código resueltas y otras que se deberán completar.

El objetivo será comprender la esencia de cómo se comportan los árboles a medida que le vamos agregando funcionalidad (o introduciendo _bugs_) para entender mejor su funcionamiento.

In [ ]:
# Puede ser necesario instalar graphviz
#!pip install graphviz

In [ ]:
# Cargamos bibliotecas necesarias
import numpy as np
import pandas as pd

from collections import Counter
import operator # Trae los operadores de Python como funciones y no infix
from IPython.display import Image, display

from graphviz import Digraph
import pydotplus

A lo largo de esta notebook usaremos objetos de distintos tipos de datos. Por simplicidad diremos que en todos los casos asumiremos que los parámetros pasados para cada función serán de los siguientes tipos:

   - el parámetro `instancias` será un `DataFrame` de `pandas`
   - el parámetro `etiquetas` será un `array` de `numpy` con valores `'Si'`, `'No'` (mismo para `etiquetas_rama_izquierda` y `etiquetas_rama_derecha`)
   - el parámetro `pregunta` será un objeto de la clase `Decision` que es definida en este mismo archivo


La clase Árbol que definiremos a continuación, consta de:

   - un constructor
   - el método `fit` para entrenarlo (a modo de sklearn)
   - el método `predict` para dada una instancia predecir su etiqueta
   - el método `score` no se encuentra implementar aún
   - métodos para visualizar y explorar el árbol



In [ ]:
# Definición de la clase árbol
from typing import Optional, Tuple


class Tree:
    def __init__(
        self,
        decision=None,  # Esto va a tener tipo Decision, una clase que vamos a definir más adelante
        left: Optional["Tree"] = None,
        right: Optional["Tree"] = None,
        labels: Optional[np.ndarray] = None,
    ):
        self.decision = decision
        self.left = left
        self.right = right

        self.data = Counter(labels) if labels is not None else None

    def fit(self, instancias: pd.DataFrame, etiquetas: np.ndarray):
        # ALGORITMO RECURSIVO para construcción de un árbol de decisión binario.

        # Suponemos que estamos parados en la raiz del árbol y tenemos que decidir cómo construirlo.
        gain, decision = encontrar_mejor_atributo_y_corte(instancias, etiquetas)

        # Criterio de corte: ¿Hay ganancia?
        if gain <= 0:
            #  Si no hay ganancia en separar, no separamos.
            self.data = Counter(etiquetas)
        else:
            # Si hay ganancia en partir el conjunto en 2
            (
                instancias_cumplen,
                etiquetas_cumplen,
                instancias_no_cumplen,
                etiquetas_no_cumplen,
            ) = partir_segun(decision, instancias, etiquetas)
            # partir devuelve instancias y etiquetas que caen en cada rama (izquierda y derecha)

            # Paso recursivo (consultar con el computadorX más cercano)
            sub_arbol_izquierdo = Tree()
            sub_arbol_izquierdo.fit(instancias_cumplen, etiquetas_cumplen)
            sub_arbol_derecho = Tree()
            sub_arbol_derecho.fit(instancias_no_cumplen, etiquetas_no_cumplen)
            # los pasos anteriores crean todo lo que necesitemos de sub-árbol izquierdo y sub-árbol derecho

            self.decision = decision
            self.left = sub_arbol_izquierdo
            self.right = sub_arbol_derecho
            self.data = Counter(etiquetas)

    def predict(self, x_t: pd.Series) -> str:
        if self.decision is None:
            if self.data["positive"] > self.data["negative"]:
                return "positive"
            else:
                return "negative"
        else:
            if self.decision.test(x_t):
                return self.left.predict(x_t)
            else:
                return self.right.predict(x_t)

    def score(self, X_test: pd.DataFrame, y_test: np.ndarray) -> float:
        # COMPLETAR
        pass

    def __repr__(self) -> str:
        return self._imprimir_arbol()

    def _imprimir_arbol(self, spacing: str = "") -> str:
        res = []
        if self.decision is None:
            res.append(spacing + f"Hoja: {dict(self.data)}")
        else:
            res.append(spacing + f"{str(self.decision)} - {dict(self.data)}")

        if self.left is not None:
            res.append(spacing + "--> True:")
            res.append(self.left._imprimir_arbol(spacing + "  "))

        if self.right is not None:
            res.append(spacing + "--> False:")
            res.append(self.right._imprimir_arbol(spacing + "  "))

        return "\n".join(res)

    def render(self) -> Digraph:
        dot = Digraph()

        self.dot_tree_aux(self, dot, prefix="")

        return dot

    def dot_tree_aux(self, subtree: "Tree", dot: Digraph, prefix: str):
        label = [
            (
                f"{subtree.decision.feature}: {subtree.decision.value}"
                if subtree.decision is not None
                else ""
            ),
            f"n={sum(subtree.data.values())}",
            str(dict(subtree.data)),
        ]
        label = "\n".join(label)
        col = "#029E3980" if subtree.data.most_common(1)[0][0] == "Si" else "#EA080080"
        dot.node(prefix + "n", label=label, shape="box", fillcolor=col, style="filled")

        if subtree.left:
            self.dot_tree_aux(subtree.left, dot, prefix + "l")
            dot.edge(prefix + "n", prefix + "ln", label="True")

        if subtree.right:
            self.dot_tree_aux(subtree.right, dot, prefix + "r")
            dot.edge(prefix + "n", prefix + "rn", label="False")

Para la decisiones en cada nodo tendremos la siguiente clase. Actualmente funciona comparando por igualdad, pero podría ser extendida en el futuro.

In [ ]:
from typing import Any

class Decision:
    def __init__(self, feature: str, value: Any, test_function=operator.eq):
        self.feature = feature
        self.value = value
        self.test_function = test_function

    def test(self, x: pd.Series) -> bool:
        # Devuelve verdadero si la instancia cumple con la pregunta
        return self.test_function(self.value, x[self.feature])

    def __repr__(self):
        return "¿Es el valor para {} igual a {}?".format(self.feature, self.value)

## Funciones a completar

Primero definir la función `gini`, que dado unas etiquetas dan el grado de impureza (ver definición en la teórica), se espera que devuelva un `float`.

In [ ]:
def gini(etiquetas: np.ndarray) -> float:
    # COMPLETAR
    impureza = 0
    return impureza

Definir la función `ganancia_gini` que dadas ciertas instancias y una posible separación entre dos ramas nos de la mejora que obtendremos al separar de esta manera. Devuelve un `float`.

In [ ]:
def ganancia_gini(etiquetas_rama_izquierda: np.ndarray, etiquetas_rama_derecha: np.ndarray) -> float:
    # COMPLETAR
    ganancia_gini = 0
    return ganancia_gini

Definir `partir_segun` que debe separar instancias y etiquetas según si cada instancia cumple o no con condición (ver método `test` de la clase `Decision`).

Para este punto se recomienda la utilizacion de máscaras de pandas (ver Notebook 01 - Herramientas).

Siguiendo con lo mencionado al inicio del NoteBook, la función debe devolver:

   - 2 `DataFrame` de `pandas` con las instancias que cumplen (`instancias_cumplen`) y las que no `instancias_no_cumplen`
   - 2 `array` de `numpy` con valores `'Si'`, `'No'`, uno con el valor de la etiqueta para las intancias que cumplen con la pregunta (`etiquetas_cumplen`) y otro con las etiquetas de las que no cumple (`etiquetas_cumplen`)


In [ ]:
def partir_segun(
    pregunta: Decision, 
    instancias: pd.DataFrame, 
    etiquetas: np.ndarray
) -> Tuple[pd.DataFrame, np.ndarray, pd.DataFrame, np.ndarray]:
    # Esta función debe separar instancias y etiquetas según si cada instancia cumple o no
    # COMPLETAR
    (instancias_cumplen,
         etiquetas_cumplen,
         instancias_no_cumplen,
         etiquetas_no_cumplen) = [], [], [], []

    return instancias_cumplen, etiquetas_cumplen, instancias_no_cumplen, etiquetas_no_cumplen

A continuación se propone una implementación para poder encontrar el mejor atributo y corte posible. Esta función devuelve un `float` y una `Decision` correspondientes al mejor atributo y corte.

In [ ]:
def encontrar_mejor_atributo_y_corte(
    instancias: pd.DataFrame, etiquetas: np.ndarray
) -> Tuple[float, Decision]:
    # Implementación Gini Gain.
    max_ganancia = 0
    mejor_pregunta = None
    for columna in instancias.columns:
        for valor in set(instancias[columna]):
            # Probando corte para atributo y valor
            pregunta = Decision(columna, valor)
            _, etiquetas_rama_izquierda, _, etiquetas_rama_derecha = partir_segun(
                pregunta, instancias, etiquetas
            )
            if len(etiquetas_rama_izquierda) == 0 or len(etiquetas_rama_derecha) == 0:
                continue

            ganancia = ganancia_gini(etiquetas_rama_izquierda, etiquetas_rama_derecha)

            if ganancia > max_ganancia:
                max_ganancia = ganancia
                mejor_pregunta = pregunta

    return max_ganancia, mejor_pregunta

Dado el siguiente dataset:

In [ ]:
from sklearn.datasets import fetch_openml
import sklearn.model_selection

tictactoe = fetch_openml(name='tic-tac-toe', version=1, as_frame=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
                    tictactoe['data'], 
                    tictactoe['target'], 
                    random_state=4, 
                    test_size=0.1)

Completar las funciones previas, entrenar y visualizar un Árbol de Decisión.

In [ ]:
arbol = Tree()
arbol.fit(X_train, y_train)
print(arbol)
arbol.render()

Para evaluar instancias en el árbol podemos elegir alguna del conjunto de testeo y evaluarlas de la siguiente manera:

In [ ]:
xs_nuevo = X_test.head(4).to_dict(orient='records')

for instancia in xs_nuevo:
    res = arbol.predict(instancia)
    print(f"Para una instancia {instancia} obtuve {res}")

¿Se obtuvieron los valores esperados? Explorar al menos 1 caso por cada rama del árbol.

## Importancia de los atributos

A continuación, implementaremos dos métodos distintos para medir la importancia de los atributos en nuestro árbol de decisión. Estos métodos nos permitirán entender mejor qué variables tienen mayor influencia en las predicciones del modelo. El primer método que implementaremos es la Importancia Gini, que se basa en la reducción de la impureza Gini en cada nodo del árbol. El segundo método es la Importancia por Permutación, que evalúa cuánto empeora el rendimiento del modelo cuando los valores de una característica se permutan aleatoriamente.

*Gini importance*

In [ ]:
def gini_importance(tree: Tree) -> dict:
    """
    Calcula la Disminución Media en la Impureza (Importancia Gini) para un árbol de decisión entrenado.

    Argumentos:
    - arbol: Una instancia de Tree entrenada

    Retorna:
    - Un diccionario que asocia los nombres de los atributos con sus puntajes de importancia Gini
    """
    
    importance = {}
    
    def recorrer_arbol(node: Tree, parent_samples: int):
        # Si es una hoja termina la recursión
        if node.decision is None:  
            return
        
        # Qué atributo se uso en ese nodo
        feature = node.decision.feature 
        
        # Si todavia no habiamos usado ese atributo para partir un nodo
        if feature not in importance:
            importance[feature] = 0
        
        # Calculamos Gini gain del corte
        gini_gain = #COMPLETAR
        
        
        # Actualizamos el valor de ese atributo
        importance[feature] = #COMPLETAR
        
        # Recorrer recursivamente los subárboles
        recorrer_arbol(node.left, parent_samples)
        recorrer_arbol(node.right, parent_samples)
    
    # Arrancamos el recorrido desde la raíz
    cantidad_total = sum(tree.data.values())
    recorrer_arbol(tree, cantidad_total)
    
    return importance

In [ ]:
gini_importance(arbol)

*Permutation importance*

In [ ]:
from typing import Callable

def permutation_importance(
    tree: Tree,
    X: pd.DataFrame,
    y: np.ndarray,
    score_func: Callable,
    n_repeats: int = 10
) -> dict:
    """
    Calcula la importancia por permutación para los atributos en un árbol entrenado.

    Argumentos:
    - tree: Instancia de Tree entrenada
    - X: DataFrame de atributos
    - y: Vector de etiquetas 
    - score_func: Función para calcular el puntaje (por ejemplo, accuracy_score, r2_score)
    - n_repeats: Número de veces para repetir el proceso de permutación

    Retorna:
    - Diccionario de importancias de los atributos
    """
    
    # Calcular el puntaje de referencia
    y_pred = np.array([tree.predict(x) for _, x in X.iterrows()])
    reference_score = score_func(y, y_pred)
    
    n_samples, n_features = X.shape
    importances = {}

    for feature in X.columns:
        feature_importance = []
        
        for _ in range(n_repeats):
            # Crear una copia de los datos
            X_corrupted = #COMPLETAR
            
            # Permutar aleatoriamente los valores en la columna feature
            X_corrupted[feature] = #COMPLETAR
            
            # Calcular el nuevo score usando los datos corrompidos
            corrupted_score = #COMPLETAR
            
            feature_importance.append(corrupted_score)
        
        # Calcular la importancia para este atributo según el algoritmo
        importances[feature] = #COMPLETAR

    return importances

In [ ]:
from sklearn.metrics import accuracy_score

permutation_importance(arbol, X_train, y_train, accuracy_score)

¿Se obtienen los mismos valores cuando se calcula sobre el conjunto de testeo?

# Opcional: Atributos continuos

La implementación anterior permite construir árboles partiendo de un dataset cuyos atributos no son continuos. Modificar dicha implementación para que soporte este tipo de atributos.
¿Qué funciones hay que modificar? 